# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from googleapi import api_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/output_data/cities.csv"
csv_df = pd.read_csv(csv_path, index_col=0)
csv_df
csv_df['City'].nunique()
clean= csv_df.drop_duplicates(subset = ['City'],keep = 'first')
clean

,City,Latitude,Longitude,Humidity,Cloudiness,Temperature,Country,Wind_Speed,Date
0,Puerto Ayora,-0.74,-90.35,91,65,73.99,EC,8.99,1607399328
1,Vanse,58.12,6.70,90,100,45.00,NO,19.91,1607399509
2,Vaini,-21.20,-175.20,78,20,82.40,TO,12.75,1607399509
3,Rikitea,-23.12,-134.97,80,3,75.99,PF,13.09,1607399350
4,Airai,-8.93,125.41,75,100,74.53,TL,3.67,1607399365
...,...,...,...,...,...,...,...,...,...
589,Tawao,4.25,117.89,74,20,86.00,MY,3.36,1607400277
590,Itarana,-19.87,-40.88,87,73,70.81,BR,3.06,1607400095
591,Touho,-20.78,165.23,70,11,81.73,NC,6.20,1607400277
592,Deputatsky,69.30,139.90,84,5,-29.38,RU,4.29,1607400132


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = clean[['Latitude','Longitude']]
weights = clean['Humidity']
gmaps.configure(api_key = api_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False, max_intensity=30,point_radius=1)
fig.add_layer(heat_layer)
fig
#plt.savefig("vacation_output/fig.png")

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
Vacation_spots = clean.loc[(clean.Temperature < 90) & (clean.Temperature > 50)
                         & (clean.Cloudiness < 34)
                         & (clean.Humidity < 30)
                         & (clean.Wind_Speed < 10)]
Vacation_spots

,City,Latitude,Longitude,Humidity,Cloudiness,Temperature,Country,Wind_Speed,Date
15,Makkah al Mukarramah,21.43,39.83,29,0,71.04,SA,7.05,1607399509
45,Yulara,-25.24,130.99,14,0,87.80,AU,9.17,1607399512
125,San Jose,37.34,-121.89,29,1,60.80,US,5.82,1607399280
129,Araouane,18.90,-3.53,28,8,65.34,ML,8.28,1607399519
141,Saint George,37.10,-113.58,20,1,51.80,US,6.93,1607399700
169,Woodland,38.68,-121.77,25,1,64.00,US,8.05,1607399702
171,Tārānagar,28.67,75.04,23,0,73.85,IN,3.83,1607399702
256,Malanville,11.87,3.39,27,0,70.02,BJ,3.58,1607399590
290,Chico,39.73,-121.84,27,1,62.60,US,4.70,1607399706
337,Opuwo,-18.06,13.84,25,15,65.23,NaN,5.61,1607399954


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create Hotels DF
hotel_df = Vacation_spots
hotel_df

,City,Latitude,Longitude,Humidity,Cloudiness,Temperature,Country,Wind_Speed,Date
15,Makkah al Mukarramah,21.43,39.83,29,0,71.04,SA,7.05,1607399509
45,Yulara,-25.24,130.99,14,0,87.80,AU,9.17,1607399512
125,San Jose,37.34,-121.89,29,1,60.80,US,5.82,1607399280
129,Araouane,18.90,-3.53,28,8,65.34,ML,8.28,1607399519
141,Saint George,37.10,-113.58,20,1,51.80,US,6.93,1607399700
169,Woodland,38.68,-121.77,25,1,64.00,US,8.05,1607399702
171,Tārānagar,28.67,75.04,23,0,73.85,IN,3.83,1607399702
256,Malanville,11.87,3.39,27,0,70.02,BJ,3.58,1607399590
290,Chico,39.73,-121.84,27,1,62.60,US,4.70,1607399706
337,Opuwo,-18.06,13.84,25,15,65.23,NaN,5.61,1607399954


In [6]:
hotel_name =[]

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "type": "lodging",
        "radius": 5000,
        "key": api_key,
}

for index, row in hotel_df.iterrows():
    try:
        lat = row['Latitude']
        lng = row['Longitude']
        
        params['location'] = f"{lat}, {lng}"
        
        response = requests.get(base_url, params = params).json()
        results = response["results"]
        print({results[0]["name"]})
        hotel_df.loc[index, "Hotel"] = results[0]["name"]
    except:
        print("No hotel found within your requested range limit")
        pass

{'Hotel Fairmont Makkah Clock Royal Tower'}


C:\Users\Matt\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Matt\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{'Desert Gardens Hotel - Ayers Rock Resort'}
{'The Westin San Jose'}
No hotel found within your requested range limit
{'Best Western Plus Abbey Inn'}
{'Hampton Inn & Suites Woodland-Sacramento Area'}
{'Jyani Plaza'}
{'DEVO HOTEL'}
{'Quality Inn Near Chico State'}
{'ABBA Guesthouse'}
{'Expeditions Ténére voyages'}
No hotel found within your requested range limit
No hotel found within your requested range limit


In [7]:
hotel_df

,City,Latitude,Longitude,Humidity,Cloudiness,Temperature,Country,Wind_Speed,Date,Hotel
15,Makkah al Mukarramah,21.43,39.83,29,0,71.04,SA,7.05,1607399509,Hotel Fairmont Makkah Clock Royal Tower
45,Yulara,-25.24,130.99,14,0,87.80,AU,9.17,1607399512,Desert Gardens Hotel - Ayers Rock Resort
125,San Jose,37.34,-121.89,29,1,60.80,US,5.82,1607399280,The Westin San Jose
129,Araouane,18.90,-3.53,28,8,65.34,ML,8.28,1607399519,NaN
141,Saint George,37.10,-113.58,20,1,51.80,US,6.93,1607399700,Best Western Plus Abbey Inn
169,Woodland,38.68,-121.77,25,1,64.00,US,8.05,1607399702,Hampton Inn & Suites Woodland-Sacramento Area
171,Tārānagar,28.67,75.04,23,0,73.85,IN,3.83,1607399702,Jyani Plaza
256,Malanville,11.87,3.39,27,0,70.02,BJ,3.58,1607399590,DEVO HOTEL
290,Chico,39.73,-121.84,27,1,62.60,US,4.70,1607399706,Quality Inn Near Chico State
337,Opuwo,-18.06,13.84,25,15,65.23,NaN,5.61,1607399954,ABBA Guesthouse


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
gmaps.configure(api_key = api_key)
coordinates = list(zip(hotel_df.Latitude, hotel_df.Longitude))
figure_layout = {
    'width' : '400px',
    'height': '300px',
    'border': '1px solid black'
    'padding': '1px'
    'margin' : '0 auto 0 auto'
}

fig = gmaps.figure(layout = figure_layout)

markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)


# Display figure
fig

SyntaxError: invalid syntax (<ipython-input-9-91f6ae1e3b0c>, line 8)